In [1]:
# https://github.com/natasha/natasha
# https://github.com/natasha/navec

from navec import Navec
from razdel import tokenize, sentenize
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from funcs import *

navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

In [2]:
df = pd.read_csv('test_corpus1.csv')
df = df[df['text'].notnull()]
sentiment_mapping = {'bad': 0, 'neutral': 1, 'good': 2}
df['target'] = df['coloring'].map(sentiment_mapping)
df['target'].value_counts()

2    2120
0    1329
1     667
Name: target, dtype: int64

## Слова в данной библиотеке кодируются следующим образом.

## Убедимся, что король - мужчина + женщина = королева

# ИМЕННО ЭТА ЯЧЕЙКА РАБОТАЕТ ДОЛГО

In [3]:
def print_nearest(vector, matrix, n=10):
    nearest_i = np.argsort(np.sum((matrix - vector)**2, axis=1))[:n]
    nearest = [navec.vocab.words[i] for i in nearest_i]
    print(nearest)

# navec.pq[navec.vocab.words.index(word)] == navec[word]
word = 'стол'
word_index = navec.vocab.words.index(word)
word_vec = navec[word]
print(f'Векторное представление слова "стол": {word_vec[:10]}')
navec_np = np.array([list(navec[word]) for word in navec.vocab.words])

Векторное представление слова "стол": [-0.2938254  -0.2968786  -0.02534253 -0.19875844  0.2802261  -0.14141266
  0.05130551 -0.18968578  0.728676   -0.6062129 ]


## Данная функция находит ближайшие слова к заданному вектору

In [4]:
print_nearest(navec['стол'], navec_np)
print_nearest(navec['король'], navec_np)
print_nearest(navec['король'] - navec['мужчина'] + navec['женщина'], navec_np)


['стол', 'стола', 'столу', 'столик', 'стул', 'столе', 'диван', 'поднос', 'столом', 'кухонный']
['король', 'принц', 'герцог', 'короля', 'генрих', 'людовик', 'монарх', 'величество', 'королю', 'император']
['король', 'королева', 'принцесса', 'короля', 'изабелла', 'королю', 'герцогиня', 'величество', 'людовик', 'королева-мать']


## Посмотрим, как работает токенизатор:

In [5]:
text = df.iloc[10, 1]
print(text)
list(tokenize(text))[:20]

Добрый день, уважаемые пользователи банковских услуг. Хочу предупредить что в данном банке не существуют единых (стандартов/гостов), сотрудники не квалифицированные на горячей линии, получить интересующую информацию возможно только в отделении банка где открыт счёт компании. Регалии современного века ввиде электронного оборота, банк не поддерживает.  Не компетентность банковских сотрудников приносит существенный урон бизнесу. Всю информацию запрашиваемую у банка, лучше получать в чате, что бы было на руках подтверждение всего деалога.


[Substring(0, 6, 'Добрый'),
 Substring(7, 11, 'день'),
 Substring(11, 12, ','),
 Substring(13, 22, 'уважаемые'),
 Substring(23, 35, 'пользователи'),
 Substring(36, 46, 'банковских'),
 Substring(47, 52, 'услуг'),
 Substring(52, 53, '.'),
 Substring(54, 58, 'Хочу'),
 Substring(59, 71, 'предупредить'),
 Substring(72, 75, 'что'),
 Substring(76, 77, 'в'),
 Substring(78, 84, 'данном'),
 Substring(85, 90, 'банке'),
 Substring(91, 93, 'не'),
 Substring(94, 104, 'существуют'),
 Substring(105, 111, 'единых'),
 Substring(112, 113, '('),
 Substring(113, 123, 'стандартов'),
 Substring(123, 124, '/')]

## Векторизивывать текст будем просто за счёт суммирования векторов всех слов, которые в него входят. Для классификации будем использвать линейную регрессию.

In [6]:
df['vector'] = df['text'].map(vectorize_text)
df_train, df_test = train_test_split(df[['text', 'target', 'coloring', 'vector']].copy(), test_size=0.3)
x_train, x_test = np.array(df_train['vector'].to_list()), np.array(df_test['vector'].to_list())
y_train, y_test = df_train['target'], df_test['target']
clf = LogisticRegression(multi_class='ovr', class_weight=(1/df_train['target'].value_counts()).to_dict(), max_iter=10000)
clf.fit(x_train, y_train)

LogisticRegression(class_weight={0: 0.0010834236186348862,
                                 1: 0.0021551724137931034,
                                 2: 0.0006693440428380187},
                   max_iter=10000, multi_class='ovr')

## В качестве метрики качества будем использовать ROC AUC

In [7]:
y_pred_train, y_pred_test = clf.predict_proba(x_train), clf.predict_proba(x_test)
df_train.insert(0, 'predictions', [x.round(3).tolist() for x in y_pred_train])
df_test.insert(0, 'predictions', [x.round(3).tolist() for x in y_pred_test])
df_test['predictions'] = [x.round(3).tolist() for x in y_pred_test]
y_pred_train_label, y_pred_test_label = y_pred_train.argmax(axis=1), y_pred_test.argmax(axis=1)
print(multiclass_roc_auc(y_train, y_pred_train))
print(multiclass_roc_auc(y_test, y_pred_test))

0.9209627708890018
0.83775178053805


## Confusion матрица

In [8]:
confusion = pd.DataFrame({'true': y_test, 'pred': y_pred_test_label}).groupby(['true', 'pred']).size() \
    .reset_index(name='fraction').pivot(index='true', columns='pred', values='fraction') \
    .reindex(index=set(sentiment_mapping.values()), columns=set(sentiment_mapping.values())).fillna(0)
confusion /= confusion.sum().sum()
print(confusion)

pred         0         1         2
true                              
0     0.185425  0.116599  0.026721
1     0.076923  0.066397  0.021053
2     0.018623  0.025101  0.463158


In [9]:
print(y_pred_train)
print(y_pred_test)

[[0.05414756 0.19533838 0.75051407]
 [0.0607772  0.27835546 0.66086735]
 [0.05957492 0.19452695 0.74589814]
 ...
 [0.06638004 0.11507028 0.81854967]
 [0.09610419 0.15371908 0.75017673]
 [0.12029712 0.1589067  0.72079617]]
[[0.01421873 0.370303   0.61547828]
 [0.02231691 0.07662563 0.90105746]
 [0.15965322 0.83877105 0.00157573]
 ...
 [0.37125496 0.32751086 0.30123419]
 [0.03961636 0.06774311 0.89264052]
 [0.55481667 0.34998675 0.09519658]]


##  Посмотрим не те тексты, которые были наиболее неправильно классифицированны

In [10]:
print_most_incorrect(df_train)
print('\n\n\n------------------------------------------------------------------------------------------\n\n\n')
print_most_incorrect(df_test)

('Я являюсь зарплатным клиентом и до этого приходили смс уведомления о том что '
 'мне одобрили кредит на 500 000+. пришел в банк, вежливая девушка Анастасия '
 'сказала ,что да, есть одобернная сумма. Стали оформлять кредит, сумма нужна '
 'была меньше 130 000 рублей. Месячный доход 65 тыс. руб. (без вычета). '
 'Девушка сказала что придется подождать до понедельника, и смогу получить '
 'кредит. В понедельник, 28 сентября прихожу в банк, жду 20 минут очереди и '
 'мне говорят что Анастасия заболела и сама мне позвонит. В этот день мне '
 'кто-то звонит и говорит что все ок - могу забирать. Прихожу в офис и говорят '
 'что Анастасии нет и что не понятно вообще без нее и что заявка на '
 'рассмотрении. Больше звонков не было. Октября Позвонил в ВТБ, оператор '
 'сказал что кредит одобрен и что я могу его получить. Пришел в Банк и мне '
 'заявляют "Кредит одобрен технически, но не нашим руководством, я оставлю '
 'комментарий Анастасии (которая куда то пропала) и она вам позвонит завтра